In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
movie_data = pd.read_csv('movies.csv')


In [3]:
movie_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movie_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [5]:
movie_data.shape

(4803, 24)

In [6]:
movie_data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [7]:
movie_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [8]:
selected_features = ['genres','keywords','cast','tagline','director','title','overview']
print(selected_features)

['genres', 'keywords', 'cast', 'tagline', 'director', 'title', 'overview']


In [9]:
for feature in selected_features:
    movie_data[feature] = movie_data[feature].fillna('')

In [10]:
combined_features = movie_data['genres']+ movie_data['keywords']+ movie_data['cast']+ movie_data['tagline']+ movie_data['director']+ movie_data['title']+ movie_data['overview']

In [11]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceEdward Burns Kerry Bish\u00e9 Ma...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [12]:
vectorizer = TfidfVectorizer()


In [13]:
feature_vectors = vectorizer.fit_transform(combined_features)


In [14]:
print(feature_vectors)

  (0, 7899)	0.1578310290038298
  (0, 1609)	0.11959809822664194
  (0, 1945)	0.05208106331752777
  (0, 34230)	0.1624432935731844
  (0, 1984)	0.03012590658276685
  (0, 31594)	0.1539703177993362
  (0, 16937)	0.13750429076715864
  (0, 4481)	0.0947430865117986
  (0, 43728)	0.14000717781279948
  (0, 4063)	0.09285674233664652
  (0, 6227)	0.058666557796787186
  (0, 28984)	0.10472117421601645
  (0, 45172)	0.1468411964952923
  (0, 31443)	0.04668262812859618
  (0, 29307)	0.14597726654100682
  (0, 43575)	0.029331122981248608
  (0, 12208)	0.1757372215443465
  (0, 22856)	0.03945275648341096
  (0, 27453)	0.13750429076715864
  (0, 32193)	0.19220324857652407
  (0, 7171)	0.12016373103312285
  (0, 274)	0.18690237186369135
  (0, 6460)	0.20866927560870166
  (0, 23099)	0.08478610106200414
  (0, 32107)	0.35781864121072393
  :	:
  (4802, 42772)	0.0675920529555793
  (4802, 31044)	0.06516100448456147
  (4802, 19463)	0.08308007988143318
  (4802, 46838)	0.08208645267269156
  (4802, 31370)	0.06100181922685574
  (48

In [15]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.05092025 0.02085813 ... 0.02355147 0.01130473 0.00647915]
 [0.05092025 1.         0.03953612 ... 0.05134641 0.03161278 0.01736785]
 [0.02085813 0.03953612 1.         ... 0.02364746 0.01160881 0.01421862]
 ...
 [0.02355147 0.05134641 0.02364746 ... 1.         0.03547579 0.01856938]
 [0.01130473 0.03161278 0.01160881 ... 0.03547579 1.         0.03378762]
 [0.00647915 0.01736785 0.01421862 ... 0.01856938 0.03378762 1.        ]]


In [16]:
print(similarity.shape)

(4803, 4803)


In [21]:
movie_name = input("Enter the name of your favorite movie : ")
list_of_all_movies = movie_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name, list_of_all_movies)
close_match = find_close_match[0]
index_of_movie = movie_data[movie_data.title == close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_movie]))
sorted_similar_movies = sorted(similarity_score, key= lambda x:x[1], reverse=True)
print("Movies suggested based on your search : \n ")


i=1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movie_data[movie_data.index ==index]['title'].values[0]
    if (i<25):
        print(i, '.', title_from_index)
        i+=1

Movies suggested based on your search : 
 
1 . The Mechanic
2 . Shadow Conspiracy
3 . Spy Game
4 . Ladyhawke
5 . The Wild Bunch
6 . DOA: Dead or Alive
7 . War
8 . Transporter 2
9 . Parker
10 . The Expendables 2
11 . Brigham City
12 . Forsaken
13 . The Last Dragon
14 . Killer Elite
15 . Kickboxer: Vengeance
16 . Safe
17 . The Expendables
18 . Woman Thou Art Loosed
19 . Crank
20 . Murderball
21 . The Raid
22 . The Man with the Golden Gun
23 . Fool's Gold
24 . The R.M.
